# FINETUNE SEGFORMER on rgb images (pytorch)

---
<a target="_blank" href="https://colab.research.google.com/drive/1FxU8SOoghUwyI-Eza_gPWllHSXyVok5k">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

https://huggingface.co/docs/transformers/model_doc/segformer

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/segformer_architecture.png"  width="600">


## Connect do GoogleDrive

---



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

## Install dependencies

---



In [ ]:
%%capture
! pip install split-folders
! pip install -U accelerate
! pip install -U transformers
! pip install evaluate
! pip install rasterio

In [ ]:
import accelerate
import transformers

print(transformers.__version__, accelerate.__version__)

from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import os
from PIL import Image
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import pandas as pd
import cv2
import numpy as np
import albumentations as aug
import random
import rasterio
from pathlib import Path
import splitfolders
import shutil

## Check GPU Ressources

---



In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

## Unzip training data

---



In [ ]:
!unzip /content/gdrive/MyDrive/FLAIR2/flair_2_dataset/sentinel_mean.zip

In [ ]:
%%capture
!mv /content/content/data /content/data

In [ ]:
import requests
import sys

# Remplacez 'YOUR_PUBLIC_LINK_HERE' par le lien public que vous avez copié.
url = 'https://drive.google.com/uc?id=1msf1Kf1sq7qPInDbtrfQg61gwlUnuyYq'

import gdown


output = 'flair_labels_train.zip'
gdown.download(url, output, quiet=False)

In [ ]:
%%capture
!unzip /content/flair_labels_train.zip

In [ ]:
# import requests
# import sys
# import gdown

# # Remplacez 'YOUR_PUBLIC_LINK_HERE' par le lien public que vous avez copié.
# url = 'https://drive.google.com/uc?id=1b3ENX6vsorKY9PgAKPOM5RGoVg0JXFRE'



# output = 'flair_aerial_train.zip'
# gdown.download(url, output, quiet=False, )

In [ ]:
# %%capture
# !unzip /content/flair_aerial_train.zip

In [ ]:
%%capture
!unzip /content/gdrive/MyDrive/FLAIR2/flair_2_dataset/flair_aerial_train.zip

## Split data for train/val

---



In [ ]:
! mkdir "/content/temp"
! mkdir "/content/data"
! mkdir "/content/data/masks"
! mkdir "/content/data/images"


# Chemin du dossier de destination
dst_folder = '/content/temp/'

# Chemin du dossier source
src_folder = '/content/flair_labels_train'
for subdir, dirs, files in os.walk(src_folder):
    for file in files:
        src_file = os.path.join(subdir, file)
        dst_file = os.path.join(dst_folder, file)
        shutil.move(src_file, dst_folder)

# Chemin du dossier source
src_folder = '/content/flair_aerial_train'
for subdir, dirs, files in os.walk(src_folder):
    for file in files:
        src_file = os.path.join(subdir, file)
        dst_file = os.path.join(dst_folder, file)
        shutil.move(src_file, dst_folder)

In [ ]:
cd temp

In [ ]:
!mv MSK*.tif /content/data/masks/
!mv  IMG*.tif /content/data/images/

splitfolders.ratio("/content/data/", seed=1337, ratio=(.999, .001), move=True) # default values

In [ ]:
# import shutil
# import os

# dossier_source = "/content/temp/content/data/sentinel"
# dossier_destination = "/content/data/sentinel"

# # Vérifier si le dossier de destination existe, sinon le créer
# if not os.path.exists(dossier_destination):
#     os.makedirs(dossier_destination)

# # Déplacer les fichiers du dossier source vers le dossier de destination
# for fichier in os.listdir(dossier_source):
#     chemin_source = os.path.join(dossier_source, fichier)
#     chemin_destination = os.path.join(dossier_destination, fichier)
#     shutil.move(chemin_source, chemin_destination)

# print("Déplacement terminé.")





In [ ]:
# import os

# dossier = "/content/data/sentinel"

# # Obtenir la liste des noms de fichiers dans le dossier
# noms_fichiers = os.listdir(dossier)

# # Compter le nombre de fichiers dans la liste
# nombre_fichiers = len(noms_fichiers)

# print(f"Le dossier {dossier} contient {nombre_fichiers} fichiers.")


## Define a class for the image segmentation dataset

---



In [ ]:
def get_data_paths (path, filter):
    for path in Path(path).rglob(filter):
        yield path.resolve().as_posix()

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
import math
class ImageSegmentationDataset(Dataset):
    """Image segmentation dataset."""


    def __init__(self, root_dir, feature_extractor,transforms):

        self.root_dir = root_dir
        self.feature_extractor = feature_extractor
        print(self.feature_extractor.size)
        self.transforms = transforms
        self.images = sorted(list(get_data_paths(Path(self.root_dir), 'IMG*.tif')), key=lambda x: int(x.split('_')[-1][:-4]))
        self.masks = sorted(list(get_data_paths(Path(self.root_dir), 'MSK*.tif')), key=lambda x: int(x.split('_')[-1][:-4]))
        self.sentinel = sorted(list(get_data_paths(Path(self.root_dir), 'SEN*.npy')), key=lambda x: int(x.split('_')[-1][:-4]))
        assert len(self.images) == len(self.masks) == len(self.sentinel), "There must be as many images as there are segmentation maps"

    def read_img(self, raster_file: str) -> np.ndarray:
        with rasterio.open(raster_file) as src_img:
            rgb = src_img.read([1,2,3]).swapaxes(0, 2).swapaxes(0, 1)
            rgb = rgb.astype(np.float32)
            return rgb

    def read_msk(self, raster_file: str) -> np.ndarray:
        with rasterio.open(raster_file) as src_msk:
            array = src_msk.read()[0]
            array = np.squeeze(array)
            return array


    def __len__(self):
        return len(self.images)


    def __getitem__(self, idx):

        image_file = self.images[idx]
        image = self.read_img(raster_file=image_file)


        sen = np.load(self.sentinel[idx])
        MEAN = np.mean(sen.squeeze(), axis=(1, 2))
        STD= np.std(sen.squeeze(), axis=(1, 2))
        MEAN = np.array(MEAN)
        STD  = np.array(STD)

        # Check for NaN values
        nan_present = any(math.isnan(x) for x in MEAN)

        if nan_present:
            MEAN = np.array([0.386759, 0.398155, 0.284823])
            STD = np.array([0.087139, 0.070850 , 0.0509254])
        else:
            pass


        test_transform = aug.Compose([   aug.Normalize(mean=MEAN, std=STD)])
        image = test_transform(image=image)['image']



        mask_file = self.masks[idx]
        segmentation_map = self.read_msk(raster_file=mask_file)
        segmentation_map[segmentation_map > 12] = 0


        encoded_inputs = self.feature_extractor(image, segmentation_map, return_tensors="pt")

        for k,v in encoded_inputs.items():
            encoded_inputs[k].squeeze_() # remove batch dimension

        return encoded_inputs

## Data augmentation with albumentation

---



In [ ]:
feature_extractor = SegformerFeatureExtractor(ignore_index=0, reduce_labels=False, do_resize=False, do_rescale=False, do_normalize=False)

root_train = '/content/temp/output/train'
root_val = '/content/temp/output/val'

train_dataset = ImageSegmentationDataset(root_dir=root_train, feature_extractor=feature_extractor,transforms='train')
valid_dataset = ImageSegmentationDataset(root_dir=root_val, feature_extractor=feature_extractor,transforms=None)

In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))

## Classes metadata

---



In [ ]:
def array_to_dict(array):
    dictionary = {}
    for i, item in enumerate(array):
        dictionary[i] = item
    return dictionary

In [ ]:
classes = ['None','building','pervious surface','impervious surface','bare soil','water','coniferous','deciduous','brushwood','vineyard','herbaceous vegetation','agricultural land','plowed land']
id2label = array_to_dict(classes)
label2id = {v: k for k, v in id2label.items()}

In [ ]:
num_labels = len(id2label)
num_labels

# Fine-tune a SegFormer model

---



## Load the model to fine-tune

In [ ]:
from transformers import SegformerForSemanticSegmentation

pretrained_model_name =  "nvidia/mit-b5" #@param {type:"string"}
model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id,
    reshape_last_stage=True,
    ignore_mismatched_sizes=True
)

## Set up the Trainer


---



In [ ]:
from transformers import TrainingArguments
import torch
epochs = 8 #@param {type:"number"}
lr = 6e-5 #@param {type:"number"}
batch_size = 4 #@param {type:"number"}
outputdir = "/content/segformer_b5_normalize_sentinel2" #@param {type:"string"}


from transformers import TrainingArguments
training_args = TrainingArguments(
    outputdir,
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",

)

## Metrics for eval

---



In [ ]:
import torch
from torch import nn
import evaluate
import multiprocessing

import os

def check_and_reconnect_drive():
    try:
        # Check if Google Drive is still connected
        os.listdir('/content/gdrive')
    except:
        # If not, reconnect it
        from google.colab import drive
        drive.mount('/content/gdrive', force_remount=True)

# Then call this function every so often in your main script




metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=False,
            # reduce_labels=feature_extractor.reduce_labels,
        )

    #add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})
    check_and_reconnect_drive()

    return metrics

## Training

---



In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,

)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("/content/gdrive/MyDrive/FLAIR2/models/segformer_b5_normalize_sentinel2")